In [1]:
import pandas as pd
import numpy as np

In [8]:
# read in csv file
all_cases = pd.read_csv('/Users/leahwallihan/Durham_school_planning/Development_cases.csv') # put in file path on your computer
all_cases.head()

,OBJECTID,A_NUMBER,A_TYPE,A_DATE,A_STATUS,A_STATUS_DATE,A_PROJECT_NAME,A_DESCRIPTION,A_USER_ID,A_CASE_PLANNER,...,DESCR_1,USER_NAME,EMAIL,ORIG_FID,CreationDate,Creator,EditDate,Editor,x,y
0,1,S1900405,FIN_EXE,11/26/2019 1:49:52 PM,APP,12/13/2019 9:45:20 AM,Longebach Recombination Survey,Recombine two lots into 1,BRYANTGR,MICHAELKL,...,Exempt Plat,Michael Klein,Michael.Klein@durhamnc.gov,1,5/22/2025 6:40:01 AM,gisproc_sys,5/22/2025 6:40:01 AM,gisproc_sys,2.024163e+06,857829.601953
1,2,HO1400256,PL_HOP,5/6/2014 3:48:46 PM,APP,5/6/2014 4:27:59 PM,Proserv Cleaner,12 Trescott Drive is the home office for the c...,MARIAHER,HSMITH,...,Home Occupation Permit,NaN,NaN,2,5/22/2025 6:40:01 AM,gisproc_sys,5/22/2025 6:40:01 AM,gisproc_sys,2.055131e+06,807863.400781
2,3,S1700288,FIN_EXE,12/11/2017 3:29:32 PM,APP,1/16/2018 10:28:22 AM,Brightleaf at the Park Tract 7 Easement,NaN,JESSICADO,JESSICADO,...,Exempt Plat,Jessica Dockery,Jessica.Dockery@durhamnc.gov,3,5/22/2025 6:40:01 AM,gisproc_sys,5/22/2025 6:40:01 AM,gisproc_sys,2.054038e+06,800861.744531
3,4,HO1100444,PL_HOP,7/29/2011 4:10:29 PM,APP,7/29/2011 4:47:29 PM,Fam or Foe Ent,Home office for musician - no onsite shows or ...,DDOTY,DDOTY,...,Home Occupation Permit,Dennis Doty,Dennis.Doty@durhamnc.gov,4,5/22/2025 6:40:01 AM,gisproc_sys,5/22/2025 6:40:01 AM,gisproc_sys,2.038546e+06,813065.083594
4,5,S1400114,FIN_MAJ,6/24/2014 8:07:07 AM,APP,7/29/2015 6:15:57 PM,Phillip and Gina Ellis Plat,Minor Subdivision,AMANDAHO,JENNIFERL,...,Final Plat,Jennifer Leaf,Jennifer.Leaf@durhamnc.gov,5,5/22/2025 6:40:01 AM,gisproc_sys,5/22/2025 6:40:01 AM,gisproc_sys,2.047560e+06,902477.569531


In [14]:
# types = A_TYPE codes we want to keep
types = ['PL_MINSP', 'PL_SSP_SM', 'PL_SSM_SM2', 'PL_MJ_COA', 'D_REZ', 'PL_CPAA', 'PL_MINPP', 'PL_MAJSP', 'PL_MAJSUP', 'PL_PPA', 'PL_MAJPP'] 
# filter that only keeps cases of specified types
filter_cases_type = all_cases[all_cases['A_TYPE'].isin(types)]

In [28]:
# status = A_STATUS codes we want to keep
status = all_cases['A_STATUS'].unique()
status = status[~np.isin(status, ['WITH', 'VOID'])]
# filter that only keeps cases of specified status
filter_cases_status = filter_cases_type[filter_cases_type['A_STATUS'].isin(status)]

In [29]:
filter_cases_status

,OBJECTID,A_NUMBER,A_TYPE,A_DATE,A_STATUS,A_STATUS_DATE,A_PROJECT_NAME,A_DESCRIPTION,A_USER_ID,A_CASE_PLANNER,...,DESCR_1,USER_NAME,EMAIL,ORIG_FID,CreationDate,Creator,EditDate,Editor,x,y
11,12,COA1700068,PL_MJ_COA,7/7/2017 12:00:00 AM,APP,9/15/2017 5:35:13 PM,1110 Wall Street,New Construction,KROSENBERG,KROSENBERG,...,Major COA,Karla Rosenberg,Karla.Rosenberg@durhamnc.gov,12,5/22/2025 6:40:01 AM,gisproc_sys,5/22/2025 6:40:01 AM,gisproc_sys,2.033476e+06,815203.547656
15,16,D1900246,PL_MINSP,7/25/2019 2:35:17 PM,APP,2/28/2020 10:45:53 AM,Tru by Hilton Hotel,"Construct 4 story, 98 room hotel, parking and ...",AMANDAHO,TREYFI,...,Level 4 - Administrative Site Plan,Trey Figueroa,Trey.Figueroa@durhamnc.gov,16,5/22/2025 6:40:01 AM,gisproc_sys,5/22/2025 6:40:01 AM,gisproc_sys,2.044848e+06,775163.066016
36,37,Z1400003,D_REZ,2/21/2014 7:42:43 AM,APP,8/5/2014 9:56:13 AM,2919 Fayetteville Street,allow retail/commercial uses in existing struc...,JOANNEG,AWOLFF,...,Zoning Map Change,Amy Wolff,Amy.Wolff@durhamnc.gov,37,5/22/2025 6:40:01 AM,gisproc_sys,5/22/2025 6:40:01 AM,gisproc_sys,2.027721e+06,805744.938281
49,50,D1200040,PL_SSP_SM,2/28/2012 9:27:43 AM,APP,4/4/2012 3:29:07 PM,Duke Student Activity Center Walkways,Provide new asphalt path from campus drive to ...,JEANETTEBR,LHODGES,...,Level 3 - Administrative Site Plan,NaN,NaN,50,5/22/2025 6:40:01 AM,gisproc_sys,5/22/2025 6:40:01 AM,gisproc_sys,2.023484e+06,818605.826562
56,57,D2200035,PL_MINSP,2/11/2022 11:24:15 AM,APP,5/15/2022 5:50:03 PM,Alexander Industrial Park - Building 3,"Proposal of 1 story ,40,950 SF of warehouse wi...",COLERE,COLERE,...,Level 4 - Administrative Site Plan,Cole Renigar,Cole.Renigar@durhamnc.gov,57,5/22/2025 6:40:01 AM,gisproc_sys,5/22/2025 6:40:01 AM,gisproc_sys,2.051101e+06,788077.191406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21270,21271,D2500109,PL_MINSP,5/6/2025 7:54:15 AM,UN_RE,5/6/2025 7:59:15 AM,Davis Park East Retail,"Three, one story multi-use buildings with asso...",VCHIRICHELLA,PEYTONBL,...,Level 4 - Administrative Site Plan,PEYTON BLANCHARD,PEYTON.BLANCHARD@DURHAMNC.GOV,21271,5/22/2025 6:40:01 AM,gisproc_sys,5/22/2025 6:40:01 AM,gisproc_sys,2.042796e+06,774410.665625
21296,21297,D2500117,PL_SSP_SM,5/12/2025 9:58:34 AM,UN_RE,5/12/2025 10:04:33 AM,Prime STQ at HUB RTP,Expansion of Building B at HUB RTP Horseshoe C...,MITRAGO,TREYFI,...,Level 3 - Administrative Site Plan,Trey Figueroa,Trey.Figueroa@durhamnc.gov,21297,5/22/2025 6:40:01 AM,gisproc_sys,5/22/2025 6:40:01 AM,gisproc_sys,2.040519e+06,780054.742578
21308,21309,D2500118,PL_MINSP,5/15/2025 9:38:22 AM,UN_RE,5/15/2025 9:50:41 AM,Yard Waste Repairs and Upgrades,Improve and upgrade existing compost facility.,FRANCISCOM,PEYTONBL,...,Level 4 - Administrative Site Plan,PEYTON BLANCHARD,PEYTON.BLANCHARD@DURHAMNC.GOV,21309,5/22/2025 6:40:01 AM,gisproc_sys,5/22/2025 6:40:01 AM,gisproc_sys,2.043918e+06,832427.332422
21314,21315,D2500120,PL_MINSP,5/19/2025 9:02:08 AM,UN_RE,5/19/2025 9:08:03 AM,Courtyards at Oak Grove II,"36 Single Family Detached dwellings, 31 town h...",ADDMGR,TREYFI,...,Level 4 - Administrative Site Plan,Trey Figueroa,Trey.Figueroa@durhamnc.gov,21315,5/22/2025 6:40:01 AM,gisproc_sys,5/22/2025 6:40:01 AM,gisproc_sys,2.060220e+06,801087.358984
